# Webscraping

## Importação das bibliotecas


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import shutil
import os
from os import path
import json
import re

In [2]:
# Driver do google chrome
driver = webdriver.Chrome()
# Acessa o site do IMDB
driver.get('https://www.imdb.com/')

In [3]:
# Acessa o menu
driver.find_element(By.ID, "imdbHeader-navDrawerOpen--desktop").click()

In [4]:
# Acessa os 250 melhores filmes
pagMelhores = driver.find_element(By.XPATH, '//*[@id="imdbHeader"]/div[2]/aside/div/div[2]/div/div[1]/span/div/div/ul/a[2]/span').click()

pagMelhoresFilmes = driver.current_url 

In [5]:
# Aqui será feito o download do poster em uma pasta 
path_to_folder = './img_posters'

if not path.exists(path_to_folder):
    os.mkdir(path_to_folder)
    
def download_poster(poster_url, title):
    response = requests.get(poster_url, stream=True)
    with open(f'{path_to_folder}/{title}.png', 'wb') as f:
        shutil.copyfileobj(response.raw, f)

    return f'{path_to_folder}/{title}.png'

In [7]:
# Aqui será feito a coleta dos dados dos filmes da página dos 250 melhores filmes
listaFilmes = []
for numFilme in range(1, 251):
    filme = {}
    titulo = driver.find_elements(By.XPATH, f'//*[@id="main"]/div/span/div/div/div[3]/table/tbody/tr[{numFilme}]/td[2]/a') # Busca o titulo do filme 
    ano = driver.find_elements(By.XPATH, f'//*[@id="main"]/div/span/div/div/div[3]/table/tbody/tr[{numFilme}]/td[2]/span') # Busca o ano do filme
    poster = driver.find_elements(By.XPATH, f'//*[@id="main"]/div/span/div/div/div[3]/table/tbody/tr[{numFilme}]/td[1]/a/img') # Busca o poster do filme
    notaIMDB = driver.find_elements(By.XPATH, f'//*[@id="main"]/div/span/div/div/div[3]/table/tbody/tr[{numFilme}]/td[3]/strong') # Busca a nota do filme

    for elemTitle in titulo:
        titulo = elemTitle.text
        filme['titulo'] = titulo

    for elemAno in ano:
        filme['ano'] = elemAno.text
        
    for elemNota in notaIMDB:
        elemNotaConv = elemNota.text 
        filme['nota imdb'] = elemNotaConv
        
    for i in poster:
        urlPoster = i.get_attribute('src')
        imagemPoster = download_poster(urlPoster, titulo)
        filme['url poster na pasta'] = imagemPoster
             
    driver.find_element(By.XPATH, f'//*[@id="main"]/div/span/div/div/div[3]/table/tbody/tr[{numFilme}]/td[2]/a').click()
    url = driver.find_elements(By.XPATH,'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[1]/div/div[1]/div/a') # Busca a url do filme
    diretores = driver.find_elements(By.XPATH,'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[1]') # Busca diretores do filme
    ListGenero = driver.find_elements(By.XPATH,'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/div[1]/div/div[2]/a/span') # Busca genero do filme

    for elemUrl in url:
        try:
            urlFilme = elemUrl.get_attribute('href')
            filme['url poster'] = urlFilme
        except:
            urlFilme = elemUrl.get_attribute('href')
            urlFilme = None
        
    for elemGenero in ListGenero:
        try:
            filme['genero'] = elemGenero.text
        except:
            elemGenero = None
        
    for elemDirect in diretores:
        try:
            diretoresFat = elemDirect.text[8:]
            filme['diretores'] = diretoresFat
        except: 
            diretoresFat = elemDirect.text[8:]
            diretoresFat = None
        
    listaFilmes.append(filme)

    driver.get(pagMelhoresFilmes) 
   
# Escreve os dados coletados em um arquivo json
with open('filmes.json', 'w') as arq:
    json.dump(listaFilmes, arq, indent=2)